In [20]:
from tensorflow.keras.datasets import cifar10
from scipy import ndimage
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

In [29]:
# code modified from  https://github.com/p-lambda/gradual_domain_adaptation

# helper functions
def get_preprocessed_cifar10():
    (train_x, train_y), (test_x, test_y) = cifar10.load_data()
    train_x, test_x = train_x / 255.0, test_x / 255.0
    train_x, train_y = shuffle(train_x, train_y)
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    #train_x = np.expand_dims(np.array(train_x), axis=-1)
    #test_x = np.expand_dims(np.array(test_x), axis=-1)
    return (train_x, train_y), (test_x, test_y)

def continually_rotate_images(xs, start_angle, end_angle):
    new_xs = []
    num_points = xs.shape[0]
    for i in range(num_points):
        angle = float(end_angle - start_angle) / num_points * i + start_angle
        img = ndimage.rotate(xs[i], angle, reshape=False)
        new_xs.append(img)
    return np.array(new_xs)

def sample_rotate_images(xs, start_angle, end_angle):
    new_xs = []
    num_points = xs.shape[0]
    for i in range(num_points):
        if start_angle == end_angle:
            angle = start_angle
        else:
            angle = np.random.uniform(low=start_angle, high=end_angle)
        img = ndimage.rotate(xs[i], angle, reshape=False)
        new_xs.append(img)
    return np.array(new_xs)

def _transition_rotation_dataset(train_x, train_y, test_x, test_y,
                                 source_angles, target_angles, inter_func,
                                 src_train_end, src_val_end, inter_end, target_end):
    assert(target_end <= train_x.shape[0])
    assert(train_x.shape[0] == train_y.shape[0])
    src_tr_x, src_tr_y = train_x[:src_train_end], train_y[:src_train_end]
    src_tr_x = sample_rotate_images(src_tr_x, source_angles[0], source_angles[1])
    src_val_x, src_val_y = train_x[src_train_end:src_val_end], train_y[src_train_end:src_val_end]
    src_val_x = sample_rotate_images(src_val_x, source_angles[0], source_angles[1])
    tmp_inter_x, inter_y = train_x[src_val_end:inter_end], train_y[src_val_end:inter_end]
    inter_x = inter_func(tmp_inter_x)
    dir_inter_x = sample_rotate_images(tmp_inter_x, target_angles[0], target_angles[1])
    dir_inter_y = np.array(inter_y)
    # dir is the "intermediate" samples directly shifted to the target angles (as comparison)
    assert(inter_x.shape == dir_inter_x.shape)
    trg_val_x, trg_val_y = train_x[inter_end:target_end], train_y[inter_end:target_end]
    trg_val_x = sample_rotate_images(trg_val_x, target_angles[0], target_angles[1])
    trg_test_x, trg_test_y = test_x, test_y
    trg_test_x = sample_rotate_images(trg_test_x, target_angles[0], target_angles[1])
    return (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x, inter_y,
            dir_inter_x, dir_inter_y, trg_val_x, trg_val_y, trg_test_x, trg_test_y)
    

def make_rotated_dataset(train_x, train_y, test_x, test_y,
                         source_angles, inter_angles, target_angles,
                         src_train_end, src_val_end, inter_end, target_end):
    inter_func = lambda x: continually_rotate_images(x, inter_angles[0], inter_angles[1])
    return _transition_rotation_dataset(
        train_x, train_y, test_x, test_y, source_angles, target_angles,
        inter_func, src_train_end, src_val_end, inter_end, target_end)
    
def tweak_one_shift(x, y, rho):
    # assign class "3" probability rho - i started with 0 but realized that doesn't composite well w rotation...
    # since 0 rotated is still 0...
    # evenly distribute among the other classes
    # y is categorical, first reverse to numerical
    y_num = np.argmax(y, axis=1)
    x_res_list = []
    y_res_list = []
    n_total = x.shape[0]
    n_classes = 10
    n_others = int((1-rho)*n_total/(n_classes-1))
    n_chosen = n_total - n_others*(n_classes-1)
    # sample from zero class
    zero_idxs = np.argwhere(y_num == 7)[:, 0]
    zero_chosen = np.random.choice(zero_idxs, n_chosen)
    x_res_list.append(x[zero_chosen])
    y_res_list.append(y[zero_chosen])
    for i in [0,1,2,3,4,5,6,8,9]:
        cur_idxs = np.argwhere(y_num == i)[:,0]
        cur_selected = np.random.choice(cur_idxs, n_others)
        x_res_list.append(x[cur_selected])
        y_res_list.append(y[cur_selected])
    x_res = np.concatenate(x_res_list,axis=0)
    y_res = np.concatenate(y_res_list,axis=0)
    # shuffle
    shuffled_idx = np.arange(x_res.shape[0])
    np.random.shuffle(shuffled_idx)
    x_res_shuffled = x_res[shuffled_idx]
    y_res_shuffled = y_res[shuffled_idx]
    return x_res_shuffled, y_res_shuffled

def resample_class_dist(x, y, p_vec): # the p_vec could be e.g. drawn from Dirichlet dist.
    n_total = x.shape[0]
    y_ordinal = np.argmax(y,axis=1)
    px_counts = p_vec * n_total
    x_res_list = []
    y_res_list = []
    for i in range(10):
        cur_idxs = np.argwhere(y_ordinal == i)[:,0]
        #print(cur_idxs)
        if len(cur_idxs) == 0:
            continue
        cur_selected = np.random.choice(cur_idxs, int(px_counts[i]))
        x_res_list.append(x[cur_selected])
        y_res_list.append(y[cur_selected])
    x_res = np.concatenate(x_res_list,axis=0)
    y_res = np.concatenate(y_res_list,axis=0)
    # shuffle
    shuffled_idx = np.arange(x_res.shape[0])
    np.random.shuffle(shuffled_idx)
    x_res_shuffled = x_res[shuffled_idx]
    y_res_shuffled = y_res[shuffled_idx]
    return x_res_shuffled, y_res_shuffled

def rotated_cifar10_60_data_func_nols():
    (train_x, train_y), (test_x, test_y) = get_preprocessed_cifar10()
    return make_rotated_dataset(
        train_x, train_y, test_x, test_y, [0.0, 2.0], [2.0, 20.0], [18.0, 20.0],
        25000, 26000, 48000, 50000)

'''
def rotated_cifar10_60_data_func_tweakone(interval, target_rho, source_rho = 0.1):
    # interval is the granularity of label shift (change rho of shift per interval)
    (train_x, train_y), (test_x, test_y) = get_preprocessed_cifar10()
    (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x, inter_y,
     dir_inter_x, dir_inter_y, trg_val_x, trg_val_y, trg_test_x, trg_test_y) = make_rotated_dataset(
        train_x, train_y, test_x, test_y, [0.0, 2.0], [2.0, 20.0], [18.0, 20.0],
        25000, 26000, 48000, 50000)
    # add label shift
    # for intermediate images we also introduce intermediate shift with granularity of interval
    n_batches = int(inter_x.shape[0]/interval)+1
    rho_list = np.linspace(source_rho, target_rho, n_batches)
    inter_x_labelshifted = []
    inter_y_labelshifted = []
    for i in range(n_batches):
        if (i+1)*interval <= inter_x.shape[0]:
            cur_x = inter_x[i*interval:(i+1)*interval]
            cur_y = inter_y[i*interval:(i+1)*interval]
        else:
            cur_x = inter_x[i*interval:]
            cur_y = inter_y[i*interval:]
        cur_shifted_x, cur_shifted_y = tweak_one_shift(cur_x, cur_y, rho_list[i])
        inter_x_labelshifted.append(cur_shifted_x)
        inter_y_labelshifted.append(cur_shifted_y)
    inter_x_ls = np.concatenate(inter_x_labelshifted, axis=0)
    inter_y_ls = np.concatenate(inter_y_labelshifted, axis=0)
    
    # dir_inter_x and dir_inter_y are for comparison (directly self-train on target)
    dir_inter_x_ls, dir_inter_y_ls = tweak_one_shift(dir_inter_x, dir_inter_y, target_rho)

    # shift all of trg_val_x, trg_val_y, trg_test_x, trg_test_y
    trg_val_x_ls, trg_val_y_ls = tweak_one_shift(trg_val_x, trg_val_y, target_rho)
    trg_test_x_ls, trg_test_y_ls = tweak_one_shift(trg_test_x, trg_test_y, target_rho)

    return (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x_ls, inter_y_ls,
            dir_inter_x_ls, dir_inter_y_ls, trg_val_x_ls, trg_val_y_ls, trg_test_x_ls, trg_test_y_ls)
'''    

def rotated_cifar10_60_data_func_dirichlet(interval, alpha, n_classes=10):
    # interval is the granularity of x|y shift
    # x|y shift is gradual, but each step we have arbitrary label shift
    # return array of all dist vecs to get oracle training
    (train_x, train_y), (test_x, test_y) = get_preprocessed_cifar10()
    (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x, inter_y,
     dir_inter_x, dir_inter_y, trg_val_x, trg_val_y, trg_test_x, trg_test_y) = make_rotated_dataset(
        train_x, train_y, test_x, test_y, [0.0, 2.0], [2.0, 20.0], [18.0, 20.0],
        25000, 26000, 48000, 50000)
    # add label shift
    # for intermediate images, each step introduce arbitrary shift
    n_batches = int(inter_x.shape[0]/interval)
    inter_x_labelshifted = []
    inter_y_labelshifted = []
    dist_store = np.zeros((n_batches+1, n_classes))
    for i in range(n_batches):
        if i == 0:
            cur_prob = np.ones(n_classes)/n_classes # start with uniform, no need to correct on first step
        else:
            cur_prob = np.random.dirichlet(np.array([alpha]*n_classes))
        dist_store[i,:] = cur_prob
        if (i+1)*interval <= inter_x.shape[0]:
            cur_x = inter_x[i*interval:(i+1)*interval]
            cur_y = inter_y[i*interval:(i+1)*interval]
        else:
            cur_x = inter_x[i*interval:]
            cur_y = inter_y[i*interval:]
        cur_shifted_x, cur_shifted_y = resample_class_dist(cur_x, cur_y, cur_prob)
        inter_x_labelshifted.append(cur_shifted_x)
        inter_y_labelshifted.append(cur_shifted_y)
    inter_x_ls = np.concatenate(inter_x_labelshifted, axis=0)
    inter_y_ls = np.concatenate(inter_y_labelshifted, axis=0)
    
    final_px = np.random.dirichlet(np.array([alpha]*n_classes))
    dist_store[n_batches,:] = final_px
    # dir_inter_x and dir_inter_y are for comparison (directly self-train on target)
    dir_inter_x_ls, dir_inter_y_ls = resample_class_dist(dir_inter_x, dir_inter_y, final_px)
    
    # shift all of trg_val_x, trg_val_y, trg_test_x, trg_test_y
    trg_val_x_ls, trg_val_y_ls = resample_class_dist(trg_val_x, trg_val_y, final_px)
    trg_test_x_ls, trg_test_y_ls = resample_class_dist(trg_test_x, trg_test_y, final_px)
    return (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x_ls, inter_y_ls,
            dir_inter_x_ls, dir_inter_y_ls, trg_val_x_ls, trg_val_y_ls, trg_test_x_ls, trg_test_y_ls, dist_store)



In [30]:
# training functions
def self_train_once(student, teacher, unsup_x, confidence_q=0.1, epochs=100, class_ws=None):
    # Do one bootstrapping step on unsup_x, where pred_model is used to make predictions,
    # and we use these predictions to update model.
    logits = teacher.predict(np.concatenate([unsup_x]))
    confidence = np.amax(logits, axis=1) - np.amin(logits, axis=1)
    alpha = np.quantile(confidence, confidence_q)
    indices = np.argwhere(confidence >= alpha)[:, 0]
    preds = np.argmax(logits, axis=1)
    preds = to_categorical(preds, num_classes=10)
    # apply class weights
    if class_ws is not None:
        # get class ws dict
        cw = {}
        for i in range(10):
            cw[i] = class_ws[i]
        student.fit(unsup_x[indices], preds[indices], epochs=epochs, class_weight = cw, verbose=False)
    else:
        student.fit(unsup_x[indices], preds[indices], epochs=epochs, verbose=False)

def soft_self_train_once(student, teacher, unsup_x, epochs=100):
    probs = teacher.predict(np.concatenate([unsup_x]))
    student.fit(unsup_x, probs, epochs=epochs, verbose=False)

def self_train(teacher, unsup_x, confidence_q=0.1, epochs=100, repeats=1,
               target_x=None, target_y=None, soft=False):
    accuracies = []
    student = teacher
    for i in range(repeats):
        if soft:
            soft_self_train_once(student, teacher, unsup_x, epochs)
        else:
            self_train_once(student, teacher, unsup_x, confidence_q, epochs)
        if target_x is not None and target_y is not None:
            _, accuracy = student.evaluate(target_x, target_y, verbose=True)
            accuracies.append(accuracy)
        teacher = student
    return accuracies, student

def gradual_self_train(teacher, unsup_x, debug_y, interval, confidence_q=0.1,
                       epochs=100, soft=False):
    upper_idx = int(unsup_x.shape[0] / interval)
    accuracies = []
    student = teacher
    for i in range(upper_idx):
        cur_xs = unsup_x[interval*i:interval*(i+1)]
        cur_ys = debug_y[interval*i:interval*(i+1)]
        # _, student = self_train(
        #     student_func, teacher, unsup_x, confidence_q, epochs, repeats=2)
        if soft:
            soft_self_train_once(student, teacher, cur_xs, epochs)
        else:
            self_train_once(student, teacher, cur_xs, confidence_q, epochs)
        _, accuracy = student.evaluate(cur_xs, cur_ys)
        accuracies.append(accuracy)
        teacher = student
    return accuracies, student

def gradual_corrected_self_train(teacher, unsup_x, debug_y, interval, dist_store, confidence_q=0.1,
                       epochs=100, soft=False):
    upper_idx = int(unsup_x.shape[0] / interval)
    accuracies = []
    student = teacher
    for i in range(upper_idx):
        cur_xs = unsup_x[interval*i:interval*(i+1)]
        cur_ys = debug_y[interval*i:interval*(i+1)]
       
        # weight by distance store
        class_ws = dist_store[i+1,:]/dist_store[i,:]
        # normalize
        class_ws = class_ws / np.mean(class_ws)
        
        if soft:
            soft_self_train_once(student, teacher, cur_xs, epochs)
        else:
            self_train_once(student, teacher, cur_xs, confidence_q, epochs, class_ws=class_ws)
        _, accuracy = student.evaluate(cur_xs, cur_ys)
        accuracies.append(accuracy)
        teacher = student
    return accuracies, student

In [31]:
# model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

def simple_softmax_conv_model():
    model = Sequential()

    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax')) 
    # compile model
    
    model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    return model



In [24]:
def rand_seed(seed):
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

def shuffle(xs, ys):
    indices = list(range(len(xs)))
    np.random.shuffle(indices)
    return xs[indices], ys[indices]

In [40]:
def exp(seed, label_shift, tweak_one_rho = None, dirichlet_alpha = None, interval=2000, epochs=30, conf_q=0.1, n_classes=10):
    # get data
    if label_shift == "NO_LS":
        (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x, inter_y, dir_inter_x, dir_inter_y,
         trg_eval_x, trg_eval_y, trg_test_x, trg_test_y) = rotated_cifar10_60_data_func_nols()
        n_batches = int(inter_x.shape[0]/interval)
        dist_store = np.ones((n_batches+1, n_classes))
    elif label_shift == "TWEAK_ONE":
        if tweak_one_rho is None:
            print("no tweak one rho!")
            return
        (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x, inter_y, dir_inter_x, dir_inter_y,
         trg_eval_x, trg_eval_y, trg_test_x, trg_test_y) = rotated_cifar10_60_data_func_tweakone(interval, tweak_one_rho)
    elif label_shift == "DIRICHLET":
        if dirichlet_alpha is None:
            print("no dirichlet alpha!")
            return
        (src_tr_x, src_tr_y, src_val_x, src_val_y, inter_x, inter_y, dir_inter_x, dir_inter_y,
         trg_eval_x, trg_eval_y, trg_test_x, trg_test_y, diststore) = rotated_cifar10_60_data_func_dirichlet(interval, dirichlet_alpha)
        
        print(diststore)
    # set seed
    rand_seed(seed)

    # Train source model
    source_model = simple_softmax_conv_model()
    source_model.fit(src_tr_x, src_tr_y, epochs=epochs, verbose=False)
    _, src_acc = source_model.evaluate(src_val_x, src_val_y)
    _, srcmodel_acc = source_model.evaluate(trg_eval_x, trg_eval_y)

    # corrected gradual self training (assuming we know label shift)
    if label_shift != "NO_LS":
        print("\n\n Gradual corrected self-training:")
        teacher = simple_softmax_conv_model()
        teacher.set_weights(source_model.get_weights())
        gradual_corrected_accuracies, student = gradual_corrected_self_train(teacher, inter_x, inter_y, interval, diststore, epochs=epochs,
            confidence_q=conf_q)
        _, acc = student.evaluate(trg_eval_x, trg_eval_y)
        gradual_corrected_accuracies.append(acc)
    
    # Train directly on target (oracle)
    print("\n\n Direct train on target:")
    target_model = simple_softmax_conv_model()
    target_model.fit(dir_inter_x, dir_inter_y, epochs=epochs, verbose=False)
    _, oracle_target_acc = target_model.evaluate(trg_eval_x, trg_eval_y)
    
    # comment this out bc we already know it's bad
    '''
    # self training on target (bad baseline)
    NUM_REPEATS = 1 # they repeat more in the paper, but small difference
    print("\n\n Direct self train on target:")
    teacher = simple_softmax_conv_model()
    teacher.set_weights(source_model.get_weights())
    target_accuracies, _ = self_train(teacher, dir_inter_x, epochs=epochs, target_x=trg_eval_x,
            target_y=trg_eval_y, repeats=NUM_REPEATS, confidence_q=conf_q)
    '''
    # Gradual self-training (paper baseline)
    print("\n\n Gradual self-training:")
    teacher = simple_softmax_conv_model()
    teacher.set_weights(source_model.get_weights())
    gradual_accuracies, student = gradual_self_train(teacher, inter_x, inter_y, interval, epochs=epochs,
            confidence_q=conf_q)
    _, acc = student.evaluate(trg_eval_x, trg_eval_y)
    gradual_accuracies.append(acc)
    

    print(f"Source model acc on target: {srcmodel_acc}")
    #print(f"direct self train on target: {target_accuracies[-1]}")
    print(f"gradual self training accuracies: {gradual_accuracies[-1]}")
    print(f"gradual labelshift corrected self training accuracies: {gradual_corrected_accuracies[-1]}")
    print(f"oracle accuracy: {oracle_target_acc}")
    return np.array([srcmodel_acc, gradual_accuracies[-1], gradual_corrected_accuracies[-1], oracle_target_acc]), diststore

In [36]:
def exp_dirichlet(seed):
    alpha = 1
    res, diststore = exp(seed, "DIRICHLET", dirichlet_alpha=alpha)
    return res, diststore
    

In [42]:
res_nols, diststore_nols = exp(1, "NO_LS")

63/63 [==============================] - 1s 19ms/step - loss: 1.0834 - accuracy: 0.6565


 Direct train on target:
63/63 [==============================] - 1s 19ms/step - loss: 0.6280 - accuracy: 0.7895


 Gradual self-training:
63/63 [==============================] - 1s 19ms/step - loss: 4.0248 - accuracy: 0.4895
Source model acc on target: 0.656499981880188
gradual self training accuracies: 0.4894999861717224


UnboundLocalError: local variable 'gradual_corrected_accuracies' referenced before assignment

In [37]:
res = np.zeros((10,5,4))
diststore = np.zeros((10,12,10))
outres = open("res.txt", "ab")
shifts = open("distr.txt", "ab")
for seed in range(10):
    res[seed,:,:], diststore[seed,:,:] = exp_dirichlet(seed)
    np.savetxt(outres, res)
    np.savetxt(shifts, diststore)
    

[[0.1        0.1        0.1        0.1        0.1        0.1
  0.1        0.1        0.1        0.1       ]
 [0.00700645 0.06043347 0.32929996 0.23529926 0.00461334 0.03359347
  0.17098958 0.13236306 0.00759186 0.01880956]
 [0.01028595 0.03045176 0.01304379 0.30528994 0.06382467 0.10863298
  0.20726798 0.12752808 0.01207499 0.12159986]
 [0.05079989 0.04618178 0.00625702 0.1186449  0.28979544 0.11569137
  0.18505634 0.14505836 0.01278833 0.02972657]
 [0.06133329 0.00073111 0.15418735 0.01424801 0.03676001 0.11951995
  0.18556773 0.11898243 0.20954469 0.09912544]
 [0.04199468 0.01292804 0.40806039 0.08835986 0.10909165 0.01680818
  0.07856265 0.00746664 0.18631029 0.05041762]
 [0.03284495 0.03081438 0.32030289 0.14763552 0.08224156 0.07279923
  0.07821055 0.17299359 0.0201879  0.04196944]
 [0.03483795 0.29212746 0.0118311  0.045789   0.08352661 0.06417046
  0.02855261 0.06022413 0.09174218 0.2871985 ]
 [0.17142503 0.08582685 0.01374764 0.04582466 0.2840542  0.13105664
  0.09631424 0.0082

In [62]:
# RESULTS (n_epochs=30)
# above was interrupted after 7 repeats, copying results here from logs above
#n_repeat * [src model, self train no correction, self train corrected, oracle]
res_shift = np.array([[0.6447895765304565,0.34769538044929504,0.4393787682056427,0.8191382884979248],
                [0.6889111995697021,0.26693427562713623,0.2599096894264221,0.7651781439781189],
                [0.6292585134506226,0.26002004742622375,0.2760521173477173,0.8151302337646484],
                [0.6175438761711121,0.33734336495399475,0.4045112729072571,0.826065182685852],
                [0.6487975716590881,0.4759519100189209,0.3491984009742737,0.7414829730987549],
                [0.6508015990257263,0.5285571217536926,0.5536072254180908,0.851703405380249],
                [0.6295739412307739,0.5298245549201965,0.4521303176879883,0.7538847327232361]])
res_mean = np.mean(res_shift, axis=0)
res_std = np.std(res_shift, axis=0)
nols_res = np.array([0.656499981880188, 0.4894999861717224, 0.789])
print("No label shift + gradual x|y rotation")
print("   src model    |            self train                  |   oracle")
print(f"{nols_res[0]:.4f}          |               {nols_res[1]:.4f}                   |  {nols_res[2]:.4f}")
print("")
print("Arbitrary label shift per step (Dirichlet alpha=1) + gradual x|y rotation, 8 repeats")
print("   src model    |    self train   | self train corrected |   oracle")
print(f"{res_mean[0]:.4f} ({res_std[0]:.4f}) | {res_mean[1]:.4f} ({res_std[1]:.4f}) |    {res_mean[2]:.4f} ({res_std[2]:.4f})   | {res_mean[3]:.4f} ({res_std[3]:.4f})")

No label shift + gradual x|y rotation
   src model    |            self train                  |   oracle
0.6565          |               0.4895                   |  0.7890

Arbitrary label shift per step (Dirichlet alpha=1) + gradual x|y rotation, 8 repeats
   src model    |    self train   | self train corrected |   oracle
0.6442 (0.0214) | 0.3923 (0.1087) |    0.3907 (0.0962)   | 0.7961 (0.0389)


In [ ]:
# RESULTS (n_epochs=100)
# no label shift exp
# 100 epochs
train on src, eval on target: 0.717
gradual self training accuracy: 0.576
oracle accuracy: 0.815

# repeat 0
# 100 epochs
Source model acc on target: 0.7094832062721252
gradual self training accuracies: 0.42900151014328003
gradual labelshift corrected self training accuracies: 0.42950326204299927
oracle accuracy: 0.7706974148750305
